In [1]:
import pandas as pd

df=pd.read_csv("data.csv")
df

,year,industry_code_ANZSIC,industry_name_ANZSIC,rme_size_grp,variable,value,unit
0,2011,A,"Agriculture, Forestry and Fishing",a_0,Activity unit,46134,COUNT
1,2011,A,"Agriculture, Forestry and Fishing",a_0,Rolling mean employees,0,COUNT
2,2011,A,"Agriculture, Forestry and Fishing",a_0,Salaries and wages paid,279,DOLLARS(millions)
3,2011,A,"Agriculture, Forestry and Fishing",a_0,"Sales, government funding, grants and subsidies",8187,DOLLARS(millions)
4,2011,A,"Agriculture, Forestry and Fishing",a_0,Total income,8866,DOLLARS(millions)
...,...,...,...,...,...,...,...
20119,2023,all,All Industries,j_Grand_Total,Total income,930995,DOLLARS(millions)
20120,2023,all,All Industries,j_Grand_Total,Total expenditure,832964,DOLLARS(millions)
20121,2023,all,All Industries,j_Grand_Total,Operating profit before tax,103616,DOLLARS(millions)
20122,2023,all,All Industries,j_Grand_Total,Total assets,2831894,DOLLARS(millions)


In [3]:
import json
import pandas as pd
import json
from sklearn.preprocessing import LabelEncoder

from load_data import CodeAutomate

label_encoder = LabelEncoder()

class Preprocess(CodeAutomate):
    def __init__(self):
        super().__init__()
        self.prepros_record={}

    def get_data(self):
        self.prepros_record={
            'onehot_encode':[],
            'label_encode':[],
            'drop_col':[]
        }
        return super().get_data()

    def process(self,col,prepros,onehot_encoder):
        col_dict=json.loads(col)
        cols=[key for key, value in col_dict.items() if value]
        # try:
        t=None
        df = pd.read_csv(f"preprocess.csv")
        if prepros == 'onehot_encode':
            for name in cols:
                encoded = onehot_encoder.fit_transform(df[[name]])
                encoded_colunms=onehot_encoder.get_feature_names_out([name])
                df.pop(name)
                feature_names=onehot_encoder.get_feature_names_out([name])
                encoder_feature=pd.DataFrame(encoded.toarray(),columns=feature_names)
                df=pd.concat([encoder_feature, df], axis=1)
                t=onehot_encoder.fit(df[[name]])
            
            temp=[]
            if self.prepros_record['onehot_encode']:
                temp=self.prepros_record['onehot_encode']
            temp.extend(cols)
            self.prepros_record['onehot_encode']=temp
            

        #     elif prepros == 'label_encode':
        #         for col in cols:
        #             if col in df.columns:
        #                 df[col] = label_encoder.fit_transform(df[col])
            
        #         temp=[]
        #         if self.prepros_record['label_encode']:
        #             temp=self.prepros_record['label_encode']
        #         temp.extend(cols)
        #         self.prepros_record['label_encode']=temp
                
        #     elif prepros == 'drop_col':
        #         df.drop(columns=cols, errors='ignore', inplace=True)

        #         temp=[]
        #         if self.prepros_record['drop_col']:
        #             temp=self.prepros_record['drop_col']
        #         temp.extend(cols)
        #         self.prepros_record['drop_col']=temp            

        bool_col=[col for col in df.columns if df[col].dtype == 'bool']
        for col_ in bool_col:
            df[col_] = df[col_].astype(int)
        df.to_csv(f"preprocess.csv",index=False)  
        records = df.to_dict("records")
        with open("preprocess_record.json", "w") as json_file:
            json.dump(self.prepros_record, json_file, indent=4) 
        return records,t
        # except:
        #     return []
        
    

ModuleNotFoundError: No module named 'load_data'

In [ ]:
import json
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
import pickle
import pandas as pd

from preprocess import Preprocess

label_encoder = LabelEncoder()
onehot_encoder=OneHotEncoder()

class Prediction(Preprocess):
    def __init__(self):
        super().__init__()
        self.onehot_encode=None

    def get_data(self):
        return super().get_data()
    
    def process(self, col, prepros):
        records,self.onehot_encode=super().process(col, prepros, onehot_encoder)
        # print(self.onehot_encode)
        # print(records)
        return records
    
    def get_json_input(self,):
        json_input={
            "number":[],
            "text":[]
        }
        try:
            df = pd.read_csv(f"data.csv")
            cols=df.columns.to_list()
            temp1={}
            for col in cols:
                if df[col].dtype == object:
                    temp2=df[col].unique().tolist()
                    temp1[col]=temp2
                else:
                    json_input["number"].append(col)
            json_input["text"]=temp1
            return json_input
        except:
            return {"message":"fail"}

    def find(self,pridict_input):
        data = json.loads(pridict_input)
        df1=pd.DataFrame(list([data.values()]),columns=list(data.keys()))
        print(self.onehot_encoder.transform(df1[['gender']]))

        # for col in df1.columns.to_list():
        #     df1[col] = label_encoder.fit_transform(df1[col])  
        # print(df1) 
        # try:
        #     with open(f"model.pkl", 'rb') as file:
        #         loaded_object = pickle.load(file)
        #     print(df1.values)
        #     predict=loaded_object.predict(df1.values)
        #     return {"prediction":str(predict[0])}
        # except:
        #     return {"message":"error"}
        
    

In [4]:
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder

label_encoder = LabelEncoder()
onehot_encoder=OneHotEncoder()

onehot=onehot_encoder.fit(df[['gender','race/ethnicity']])
test=onehot.transform(df[['gender','race/ethnicity']][:4])
test.toarray()

KeyError: "None of [Index(['gender', 'race/ethnicity'], dtype='object')] are in the [columns]"

In [ ]:
import pandas as pd
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import FunctionTransformer, LabelEncoder
from sklearn.pipeline import Pipeline

# Sample data with multiple categorical columns
data =pd.read_csv('data.csv')
data=data.drop(["math score","reading score","writing score"],axis=1)
def label_encode(df):
    label_encoders = {}
    for column in df.columns:
        le = LabelEncoder()
        df[column] = le.fit_transform(df[column])
        label_encoders[column] = le
    return df

# Use FunctionTransformer to apply the custom function in ColumnTransformer
preprocessor = ColumnTransformer(
    transformers=[
        ('cat', FunctionTransformer(label_encode), data.columns)  # Apply LabelEncoder
    ])

# Transform the data
encoded_data = preprocessor.fit_transform(data.copy())

# Convert back to a DataFrame for readability
encoded_df = pd.DataFrame(encoded_data, columns=data.columns)

print(encoded_df)

In [7]:
encoded_data = preprocessor.fit_transform(data.iloc[0:5,:])
encoded_df = pd.DataFrame(encoded_data, columns=data.columns)

print(encoded_df)
print(data.iloc[0:5,:])

NameError: name 'preprocessor' is not defined

In [9]:
import pandas as pd
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import FunctionTransformer, LabelEncoder, OneHotEncoder

# Sample data with multiple categorical columns
data = pd.read_csv("data.csv")

# Define the label encoding function (for specific columns in cols1)
def label_encode(X):
    X = pd.DataFrame(X)  # Convert to DataFrame for ease
    for column in X.columns:
        le = LabelEncoder()
        X[column] = le.fit_transform(X[column])
    return X

# OneHotEncoder will be used directly from sklearn for columns in cols2

# Specify the columns for label encoding and one-hot encoding
cols1 = ['gender']    # Columns for Label Encoding
cols2 = ['race/ethnicity']   # Columns for One-Hot Encoding

# Use ColumnTransformer to apply different encodings
preprocessor = ColumnTransformer(
    transformers=[
        ('cat_label', FunctionTransformer(label_encode), cols2),  # Apply label encoding to cols1
        ('cat_onehot', OneHotEncoder(), cols1)  # Apply OneHotEncoder to cols2
    ])

# Apply the transformation
encoded_data = preprocessor.fit_transform(data.iloc[-10:,:2])

# Convert encoded_data to DataFrame for better readability
# OneHotEncoder output is a sparse matrix by default, so convert it to dense
encoded_df = pd.DataFrame(encoded_data)

print(encoded_df)


ValueError: A given column is not a column of the dataframe

In [11]:
data.iloc[-10:,:2]

,year,industry_code_ANZSIC
20114,2023,S
20115,2023,all
20116,2023,all
20117,2023,all
20118,2023,all
20119,2023,all
20120,2023,all
20121,2023,all
20122,2023,all
20123,2023,all


In [13]:
import pandas as pd
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import FunctionTransformer, LabelEncoder, OneHotEncoder

# Sample data with multiple categorical columns
data = pd.read_csv('data.csv')

# Define the label encoding function (for specific columns in cols1)
def label_encode(X):
    X = pd.DataFrame(X)  # Convert to DataFrame for ease
    for column in X.columns:
        le = LabelEncoder()
        X[column] = le.fit_transform(X[column])
    return X

# Specify the columns for label encoding and one-hot encoding
cols1 = ['gender']    # Columns for Label Encoding
cols2 = []   # Columns for One-Hot Encoding

# Use ColumnTransformer to apply different encodings
preprocessor = ColumnTransformer(
    transformers=[
        ('cat_label', FunctionTransformer(label_encode), cols1),  # Apply label encoding to cols1
        ('cat_onehot', OneHotEncoder(), cols2)  # Apply OneHotEncoder to cols2
    ])

# Fit the transformer on the data
preprocessor.fit(data)

# Get the names of the one-hot encoded columns
onehot_columns = preprocessor.transformers_[1][1].get_feature_names_out(cols2)

# Combine label-encoded columns and one-hot encoded column names
final_columns = cols1 + list(onehot_columns)

# Apply the transformation
encoded_data = preprocessor.transform(data.iloc[10:12,:])

# Convert encoded_data to DataFrame for better readability
encoded_df = pd.DataFrame(encoded_data, columns=final_columns)

print(encoded_df)


ValueError: A given column is not a column of the dataframe

In [3]:
import pandas as pd
df1=pd.read_csv("preprocess.csv")
df1

,year,industry_code_ANZSIC,industry_name_ANZSIC,rme_size_grp,variable,value,unit
0,2011,A,"Agriculture, Forestry and Fishing",a_0,Activity unit,46134,COUNT
1,2011,A,"Agriculture, Forestry and Fishing",a_0,Rolling mean employees,0,COUNT
2,2011,A,"Agriculture, Forestry and Fishing",a_0,Salaries and wages paid,279,DOLLARS(millions)
3,2011,A,"Agriculture, Forestry and Fishing",a_0,"Sales, government funding, grants and subsidies",8187,DOLLARS(millions)
4,2011,A,"Agriculture, Forestry and Fishing",a_0,Total income,8866,DOLLARS(millions)
...,...,...,...,...,...,...,...
20119,2023,all,All Industries,j_Grand_Total,Total income,930995,DOLLARS(millions)
20120,2023,all,All Industries,j_Grand_Total,Total expenditure,832964,DOLLARS(millions)
20121,2023,all,All Industries,j_Grand_Total,Operating profit before tax,103616,DOLLARS(millions)
20122,2023,all,All Industries,j_Grand_Total,Total assets,2831894,DOLLARS(millions)


In [7]:
df1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20124 entries, 0 to 20123
Data columns (total 7 columns):
 #   Column                Non-Null Count  Dtype 
---  ------                --------------  ----- 
 0   year                  20124 non-null  int64 
 1   industry_code_ANZSIC  20124 non-null  object
 2   industry_name_ANZSIC  20124 non-null  object
 3   rme_size_grp          20124 non-null  object
 4   variable              20124 non-null  object
 5   value                 20124 non-null  object
 6   unit                  20124 non-null  object
dtypes: int64(1), object(6)
memory usage: 1.1+ MB


In [66]:
des=df1.info()
print(des)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20124 entries, 0 to 20123
Data columns (total 7 columns):
 #   Column                Non-Null Count  Dtype 
---  ------                --------------  ----- 
 0   year                  20124 non-null  int64 
 1   industry_code_ANZSIC  20124 non-null  object
 2   industry_name_ANZSIC  20124 non-null  object
 3   rme_size_grp          20124 non-null  object
 4   variable              20124 non-null  object
 5   value                 20124 non-null  object
 6   unit                  20124 non-null  object
dtypes: int64(1), object(6)
memory usage: 1.1+ MB
None


In [68]:
import pandas as pd
import io

def get_dataframe_info(df):
    buffer = io.StringIO()
    df.info(buf=buffer)  # Capture the output of df.info()
    return buffer.getvalue()  # Return it as a string

# Example DataFrame
data = {"A": [1, 2, 3], "B": [4.5, 5.5, 6.5]}
df = pd.DataFrame(data)

info_str = get_dataframe_info(df)
print(info_str)  # Prints the DataFrame info as a string


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3 entries, 0 to 2
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   A       3 non-null      int64  
 1   B       3 non-null      float64
dtypes: float64(1), int64(1)
memory usage: 180.0 bytes

